This will launch the Streamlit web application, which can be accessed at http://localhost:8501 in your web browser. You can upload an ECG image, and the application will display the original and processed images along with the predicted class

To run the application, navigate to the directory containing ecg_app.py in your terminal and execute:



In [2]:
import streamlit as st
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image

In [3]:


# Utility functions (preprocess_image, prepare_image_for_model, load_model, predict)

def preprocess_image(image_path):
    # Load the image
    image = cv2.imread(image_path)
    original_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
    
    # Enhance contrast using histogram equalization
    equalized_image = cv2.equalizeHist(blurred_image)
    
    # Detect edges using Canny edge detector
    edges = cv2.Canny(equalized_image, 100, 200)
    
    # Plot the original and processed images
    fig, axes = plt.subplots(2, 3, figsize=(10, 8))
    axes[0, 0].set_title('Original Image')
    axes[0, 0].imshow(original_image)
    axes[0, 1].set_title('Grayscale Image')
    axes[0, 1].imshow(gray_image, cmap='gray')
    axes[0, 2].set_title('Gaussian Blur')
    axes[0, 2].imshow(blurred_image, cmap='gray')
    axes[1, 0].set_title('Histogram Equalization')
    axes[1, 0].imshow(equalized_image, cmap='gray')
    axes[1, 1].set_title('Canny Edges')
    axes[1, 1].imshow(edges, cmap='gray')
    for ax in axes.flatten():
        ax.axis('off')
    plt.tight_layout()
    st.pyplot(fig)
    
    # Convert processed image to 3-channel format
    processed_image = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
    return processed_image

def prepare_image_for_model(image_array):
    image_tensor = np.expand_dims(image_array, axis=0)
    image_tensor = np.expand_dims(image_tensor, axis=-1)
    return image_tensor

def load_model(model_path):
    return tf.keras.models.load_model(model_path)

def predict(model, image_tensor):
    prediction = model.predict(image_tensor)
    return np.argmax(prediction, axis=1)[0]

# Streamlit app
st.title('ECG Image Classification')

# Upload image
uploaded_file = st.file_uploader("Choose an ECG image...", type=["jpg", "jpeg", "png"])
if uploaded_file is not None:
    image = Image.open(uploaded_file)
    image_path = f"uploads/{uploaded_file.name}"
    image.save(image_path)
    
    # Process and display the image
    processed_image = preprocess_image(image_path)
    
    # Prepare image for model
    image_tensor = prepare_image_for_model(processed_image)
    
    # Load the model and make a prediction
    model = load_model('LSTM')  # Ensure this is the correct path to your model
    prediction = predict(model, image_tensor)
    
    class_names = {0: 'Abnormal', 1: 'Covid', 2: 'Normal'}
    predicted_class = class_names[prediction]
    
    st.write(f'Predicted class: {predicted_class}')


2024-05-19 08:59:25.201 
  command:

    streamlit run C:\Users\Deepak\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
